In [7]:
# from concurrent.futures import process
# from pinecone import Pinecone, ServerlessSpec
# from dotenv import load_dotenv

# pc = Pinecone(api_key=process.env.PINECONE_API_KEY)

In [8]:
# pc.create_index(
#     name="quickstart",
#     dimension=2, # Replace with your model dimensions
#     metric="cosine", # Replace with your model metric
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ) 
# )